In [1]:
using JuMP
using Ipopt
using Random
using Printf
using LinearAlgebra

In [2]:
## Function to solve a small instance of the deterministic model
function deterministic_model_small()

    ##################### PROBLEM DATA FOR SMALLER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 100                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################
    
    ## We first solve the problem formulation directly without ADMM
    tini  = time()                                      # Start timer
    model = Model(with_optimizer(Ipopt.Optimizer))      # We use Ipopt solver to compare with ADMM   

    ## Variables
    @variable(model, x[i in I] >= 0)                    # Reserved capacity variables
    @variable(model, y[i in I, j in J, s in S] >= 0)    # Demand fulfilment variables
    @variable(model, u[j in J, s in S] >= 0)            # Unfulfilled demand variables

    ## Objective: Minimize the total phase 1 + phase 2 costs over all scenarios
    @objective(model, Min,
          sum(c[i]*x[i] for i in I) +
          sum(p[s]*f[i,j]*y[i,j,s] for s in S, i in I, j in J) + 
          sum(p[s]*q[j]*u[j,s] for s in S, j in J))

    ## Constraints
    @constraint(model, [i in I], x[i] <= b[i])          # Max capacity constraint
    @constraint(model, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost) constraint

    ## Capacity reserve limit constraint for each supplier i in each scenario s
    @constraint(model, [i in I, s in S], sum(y[i,j,s] for j in J) <= x[i])

    ## Demand balance constraint (u[j,s] is unfulfilled demand of client j in scenario s)
    @constraint(model, DemBal[j in J, s in S], sum(y[i,j,s] for i in I) + u[j,s] == d[j,s])

    optimize!(model)                                    # Solve the problem

    status = termination_status(model)                  # Solution status
    println(status)                                     # Print status
    
    ## Stop timer
    tend = time() - tini
   
    ## Return solution vector x, costs c, and solution time tend
    return (c, x, tend)
    
end

deterministic_model_small (generic function with 1 method)

In [3]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the small deterministic instance. 
(c, x, tend) = deterministic_model_small()

xsol = value.(x)                          # Get optimal x values (reserved capacities)
xsol = round.(xsol.data, digits = 2)      # Round to 2 decimals 
fval = dot(c, xsol)                       # Optimal cost of reserved capacities

## Solution time of small deterministic model
println("\nSolution time: ", tend, "\n")

## Print optimal solution 
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    63000
Number of nonzeros in inequality constraint Jacobian.:    62040
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    63020
                     variables with only lower bounds:    63020
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ


Solution time: 53.71412014961243

Optimal solution:

x[ 1] =    0.00
x[ 2] =   29.00
x[ 3] =    0.00
x[ 4] =   42.00
x[ 5] =   12.40
x[ 6] =    0.00
x[ 7] =   73.00
x[ 8] =    0.00
x[ 9] =   64.00
x[10] =   61.00
x[11] =    0.00
x[12] =   29.00
x[13] =   85.00
x[14] =    0.00
x[15] =    0.00
x[16] =    0.00
x[17] =    0.00
x[18] =    0.00
x[19] =   36.00
x[20] =    0.00

Optimal cost of reserved capacities: 3000.0


In [4]:
## Function to solve a large instance of the deterministic model
function deterministic_model_large()

    ##################### PROBLEM DATA FOR LARGER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 500                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################
    
    ## We first solve the problem formulation directly without ADMM
    tini  = time()                                      # Start timer
    model = Model(with_optimizer(Ipopt.Optimizer))      # We use Ipopt solver to compare with ADMM   

    ## Variables
    @variable(model, x[i in I] >= 0)                    # Reserved capacity variables
    @variable(model, y[i in I, j in J, s in S] >= 0)    # Demand fulfilment variables
    @variable(model, u[j in J, s in S] >= 0)            # Unfulfilled demand variables

    ## Objective: Minimize the total phase 1 + phase 2 costs over all scenarios
    @objective(model, Min,
          sum(c[i]*x[i] for i in I) +
          sum(p[s]*f[i,j]*y[i,j,s] for s in S, i in I, j in J) + 
          sum(p[s]*q[j]*u[j,s] for s in S, j in J))

    ## Constraints
    @constraint(model, [i in I], x[i] <= b[i])          # Max capacity constraint
    @constraint(model, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost) constraint

    ## Capacity reserve limit constraint for each supplier i in each scenario s
    @constraint(model, [i in I, s in S], sum(y[i,j,s] for j in J) <= x[i])

    ## Demand balance constraint (u[j,s] is unfulfilled demand of client j in scenario s)
    @constraint(model, DemBal[j in J, s in S], sum(y[i,j,s] for i in I) + u[j,s] == d[j,s])

    optimize!(model)                                    # Solve the problem

    status = termination_status(model)                  # Solution status
    println(status)                                     # Print status 
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x, tend)
end

deterministic_model_large (generic function with 1 method)

In [5]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the large deterministic model
(c, x, tend) = deterministic_model_large()

xsol = value.(x)                          # Get optimal x values (reserved capacities)
xsol = round.(xsol.data, digits = 2)      # Round to 2 decimals 
fval = dot(c, xsol)                       # Optimal cost of reserved capacities

## Solution time of the large deterministic model
println("\nSolution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   315000
Number of nonzeros in inequality constraint Jacobian.:   310040
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:   315020
                     variables with only lower bounds:   315020
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    15000
Total number of inequality constraints...............:    10021
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:    10021

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 


Solution time: 352.04752492904663

Optimal solution:

x[ 1] =    0.00
x[ 2] =   92.00
x[ 3] =    0.00
x[ 4] =   51.00
x[ 5] =    0.00
x[ 6] =    0.00
x[ 7] =   77.00
x[ 8] =    0.00
x[ 9] =   22.00
x[10] =  100.00
x[11] =    0.00
x[12] =    4.00
x[13] =   29.00
x[14] =    0.00
x[15] =    0.00
x[16] =    0.00
x[17] =    0.00
x[18] =    0.00
x[19] =   47.00
x[20] =    0.00

Optimal cost of reserved capacities: 3000.0


In [11]:
#######################################################################################
####           NOTE: RESTART THE KERNEL BEFORE TESTING THE ADMM CODE.             #####  
####                 MEMORY MAY BECOME AN ISSUE OTHERWISE.                        ##### 
#######################################################################################
#### ADMM using Ipopt (other available solvers do not support quadratic objective) ####
#######################################################################################
using Distributed               # For parallel computing 
using Random                    # For controlling randomness
using Printf                    # For C-style printf macro
using SharedArrays              # For shared memory access

addprocs(10)                     # Use 4 parallel threads (add 4 new workers)

@everywhere using LinearAlgebra # For norm() function 
@everywhere using JuMP          # NOTE: The macro @everywhere makes the loaded packages available to all threads
@everywhere using Ipopt         # Solver capable of solving quadratic objectives

In [27]:
## Function to solve a small instance of the deterministic model.
##
## TODO: Fill in the missing code inside the function. Compare your solution 
##       to that of the small deterministic model. These should be almost equal.
##
## Input argument: penalty parameter ρ
function admm_model_small(ρ::Float64)
    
    ##################### PROBLEM DATA FOR SMALLER INSTANCE ########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 100                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################

    tini = time()                      # Start timer
    
    ## NOTE: ADMM approach starts from here. Complete the missing code parts that
    ##       are requested. Code lines to be completed are marked as follows:
    ##       TODO: Complete the following expression. (For example)

    ## Problem parameters and memory allocation. NOTE: Compare with Exercise 10.1
    x_s  = SharedArray(zeros(nI, nS))  # Store each x vector in (x,y)-steps of each scenario
    v_s  = SharedArray(zeros(nI, nS))  # Store each v vector in v-steps of each scenario
    z    = SharedArray(zeros(nI))      # Store z vectors at each ADNM iteration
    
    ## Main loop with 200 ADMM iterations
    for k = 1:200
        ## Loop for solving each (x,y) step separately for each scenario s in S
        ## NOTE: Compare with Exercise 10.1
        ## NOTE: The macro @distributed allocates scenario subproblems to available
        ##       threads. The macro @sync ensures that all scenario subproblems
        ##       are solved before proceeding with the code after the for loop
        @sync @distributed for s in S   
            
            ## Model to solve (x,y)-step of the current scenario subproblem s
            scen_m = Model(with_optimizer(Ipopt.Optimizer)) 
            
            @variable(scen_m, x[i in I] >= 0)                    # Reserved capacity variables
            @variable(scen_m, y[i in I, j in J] >= 0)            # Demand fulfilment variables
            @variable(scen_m, u[j in J] >= 0)                    # Unfulfilled demand variables

            @constraint(scen_m, [i in I], x[i] <= b[i])          # Max capacity constraint
            @constraint(scen_m, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost)
            
            ## Capacity reserve limit constraint for each supplier i
            @constraint(scen_m, [i in I], sum(y[i,j] for j in J) <= x[i])
            
            ## Demand balance constraint (u[j] is unfulfilled demand of client j)
            @constraint(scen_m, [j in J], sum(y[i,j] for i in I) == d[j,s] - u[j])

            #### TODO: Complete this objective to compute (x, y) - step for the
            ####       current scenario. Compare with Exercise 10.1
            @objective(scen_m, Min,
                sum(c[i]*x[i] + v_s[i,s]*x[i] for i in I) + 
                sum(f[i,j]*y[i,j] for i in I, j in J) + 
                sum(q[j]*u[j] for j in J) + 
                #sum((x[i]-z[i]) for i in I) +  # Before penalty?
                (ρ/2.0)*sum((x[i]-z[i])^2 for i in I) # Norm
            )

            ## Redirect stdout to omit unnecessary output from workers
            tmp = stdout           
            redirect_stdout()

            ## Solve the (x,y) step for the current scenario
            optimize!(scen_m)

            ## Restore stdout back to normal 
            redirect_stdout(tmp)  

            ## Store the value of x for the current scenario
            x_s[:,s] = value.(x[:])
        end

        ## Compute primal and dual residuals. We use SharedArray to exploit parallelism
        tol = SharedArray(zeros(nS))
        
        #### TODO: Complete the computation of the residual for each s inside the for loop.
        ####       Compare with Exercise 10.1
        @sync @distributed for s in S
             tol[s] = p[s]*ρ*norm(x_s[:,s] - z)
        end
        
        ## Total residual = sum of subproblem residuals
        tol = sum(tol[s] for s in S)

        ## Print current progress
        println("iteration: ", k ,"/ residual: ", tol)
        
        ## Stopping condition: if primal + dual residual is small enough
        if tol < 1e-1
            ## Stop timer
            tend = time() - tini
            return (c, x_s, tend, k)
        end

        #### TODO: Complete the z-step. Compare with Exercise 10.1
        z = x_s*p # Done

        #### TODO: Complete the v-steps for each scenario s.
        ####       Compare with exercise 10.1
        #@sync for s in S
        #    println("s")
        #    println(size(v_s[:,s]))
        #    println(size(x_s[s]))
        #    println(size(z))
        #    println(v_s[:,s] + ρ*(x_s[:,s] - z))
        #    println("emd")
        #end
        @sync @distributed for s in S
            v_s[:,s] = v_s[:,s] + ρ*(x_s[:, s] - z)# Done v_s should be -1 for s
        end
    end
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x_s, tend, 200)    
end

admm_model_small (generic function with 1 method)

In [8]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the small ADMM model

## Try different values of penalty parameter ρ ∈ [1.0, 100.0]
ρ = 100.0                                   

(c, x_s, tend, k) = admm_model_small(ρ)

xsol = round.(x_s[:,1], digits = 2)     # Get optimal x values (reserved capacities)
fval = dot(c, x_s[:,1])                 # Optimal cost of reserved capacities

## Solution time of ADMM:
println("\nADMM Solution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nCost of reserved capacity: ", fval)

iteration: 1/ residual: 10128.151631679988
iteration: 2/ residual: 3897.0472988761526
iteration: 3/ residual: 2094.366692761772
iteration: 4/ residual: 1357.785693925109
iteration: 5/ residual: 659.8889738082378
iteration: 6/ residual: 579.0866346900767
iteration: 7/ residual: 372.44775312180843
iteration: 8/ residual: 329.8535794275483
iteration: 9/ residual: 238.30283218384054
iteration: 10/ residual: 166.70059096506833
iteration: 11/ residual: 115.57367982101998
iteration: 12/ residual: 59.9671000337686
iteration: 13/ residual: 34.30364386423431
iteration: 14/ residual: 32.9878603570172
iteration: 15/ residual: 11.328065122191084
iteration: 16/ residual: 9.531573608007855
iteration: 17/ residual: 7.899776623998097
iteration: 18/ residual: 7.090028215313696
iteration: 19/ residual: 6.682110300726924
iteration: 20/ residual: 6.2833976993005765
iteration: 21/ residual: 5.964693815624256
iteration: 22/ residual: 5.674379076735991
iteration: 23/ residual: 5.353801278021653
iteration: 24/

In [72]:
## Function to solve a large instance of the deterministic model.
##
## TODO: Fill in the missing code inside the function. Compare your solution 
##       to that of the large deterministic model. These should be almost equal.
##
## Input argument: penalty parameter ρ
function admm_model_large(ρ::Float64)
    
    ##################### PROBLEM DATA FOR LARGER INSTANCE #########################
    Random.seed!(1)                   # Control random number generation
    nI = 20                           # Number of suppliers
    nJ = 30                           # Number of clients
    nS = 500                          # Number of scenarios
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    #### Generate random data for the problem
    c = rand(5:20, nI)                # Unit capacity costs of suppliers
    d = rand(nJ,nS).*rand(10:50, nJ)  # Client demands in all scenarios
    q = rand(5000:10000, nJ)          # Unit costs of unfulfilled demand
    p = ones(nS).*1/nS                # Scenario probabilities
    f = rand(3:45, (nI,nJ))           # Unit costs to fulfil demands
    b = rand(20:100, nI)              # Max supplier capacities
    B = 3000                          # Max budget (cost) for capacity acquisition
    ################################################################################

    tini = time()                      # Start timer
    
    ## NOTE: ADMM approach starts from here. Complete the missing code parts that
    ##       are requested. Code lines to be completed are marked as follows:
    ##       TODO: Complete the following expression. (For example)

    ## Problem parameters and memory allocation. NOTE: Compare with Exercise 10.1
    x_s  = SharedArray(zeros(nI, nS))  # Store each x vector in (x,y)-steps of each scenario
    v_s  = SharedArray(zeros(nI, nS))  # Store each v vector in v-steps of each scenario
    z    = SharedArray(zeros(nI))      # Store z vectors at each ADNM iteration
    
    ## Main loop with 200 ADMM iterations
    for k = 1:200
        ## Loop for solving each (x,y) step separately for each scenario s in S
        ## NOTE: Compare with Exercise 10.1
        ## NOTE: The macro @distributed allocates scenario subproblems to available
        ##       threads. The macro @sync ensures that all scenario subproblems
        ##       are solved before proceeding with the code after the for loop
        @sync @distributed for s in S   
            
            ## Model to solve (x,y)-step of the current scenario subproblem s
            scen_m = Model(with_optimizer(Ipopt.Optimizer)) 
            
            @variable(scen_m, x[i in I] >= 0)                    # Reserved capacity variables
            @variable(scen_m, y[i in I, j in J] >= 0)            # Demand fulfilment variables
            @variable(scen_m, u[j in J] >= 0)                    # Unfulfilled demand variables

            @constraint(scen_m, [i in I], x[i] <= b[i])          # Max capacity constraint
            @constraint(scen_m, sum(c[i]*x[i] for i in I) <= B)  # Max capacity budget (cost)
            
            ## Capacity reserve limit constraint for each supplier i
            @constraint(scen_m, [i in I], sum(y[i,j] for j in J) <= x[i])
            
            ## Demand balance constraint (u[j] is unfulfilled demand of client j)
            @constraint(scen_m, [j in J], sum(y[i,j] for i in I) == d[j,s] - u[j])

            #### TODO: Complete this objective to compute (x, y) - step for the
            ####       current scenario. Compare with Exercise 10.1
            @objective(scen_m, Min,
                sum(c[i]*x[i] + v_s[i,s]*x[i] for i in I) + 
                sum(f[i,j]*y[i,j] for i in I, j in J) + 
                sum(q[j]*u[j] for j in J) + 
                #sum((x[i]-z[i]) for i in I) +  # Before penalty?
                (ρ/2.0)*sum((x[i]-z[i])^2 for i in I) # Norm
            )

            ## Redirect stdout to omit unnecessary output from workers
            tmp = stdout           
            redirect_stdout()

            ## Solve the (x,y) step for the current scenario
            optimize!(scen_m)

            ## Restore stdout back to normal 
            redirect_stdout(tmp)  

            ## Store the value of x for the current scenario
            x_s[:,s] = value.(x[:])
        end

        ## Compute primal and dual residuals. We use array to exploit parallelism
        tol = SharedArray(zeros(nS))
        
        #### TODO: Complete the computation of the residual for each s inside the for loop.
        ####       Compare with Exercise 10.1
        @sync @distributed for s in S
             tol[s] = p[s]*ρ*norm(x_s[:,s] - z)
        end
        
        ## Total residual = sum of subproblem residuals
        tol = sum(tol[s] for s in S)

        ## Print current progress
        println("iteration: ", k ,"/ residual: ", tol)
        
        ## Stopping condition: if primal + dual residual is small enough
        if tol < 1e-1
            ## Stop timer
            tend = time() - tini
            return (c, x_s, tend, k)
        end

        #### TODO: Complete the z-step. Compare with Exercise 10.1
        z = sum(p[s]*x_s[:,s] for s in S)

        #### TODO: Complete the v-steps for each scenario s.
        ####       Compare with exercise 10.1
        @sync @distributed for s in S
            v_s[:,s] = v_s[:,s] + ρ*(x_s[:, s] - z)
        end
    end
    
    ## Stop timer
    tend = time() - tini
    
    ## Return solution vector x, costs c, and solution time tend
    return (c, x_s, tend, 200)    
end

Opening in existing browser session.


admm_model_large (generic function with 1 method)

In [10]:
## NOTE: Run this cell twice, the first run is always slower due to "compilation" (Julia uses JIT compilation)
## Solve the large ADMM model

## Try different values of penalty parameter ρ ∈ [1.0, 100.0]
ρ = 100.0                                   

(c, x_s, tend, k) = admm_model_large(ρ)

xsol = round.(x_s[:,1], digits = 2)     # Get optimal x values (reserved capacities)
fval = dot(c, x_s[:,1])                 # Optimal cost of reserved capacities

## Solution time of ADMM:
println("\nADMM Solution time: ", tend, "\n")

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol)
    @printf("x[%2d] = %7.2f\n", i, xsol[i])
end

## Print optimal cost of reserved capacities
println("\nCost of reserved capacity: ", fval)

iteration: 1/ residual: 9713.373880009747
iteration: 2/ residual: 3847.767198231959
iteration: 3/ residual: 1889.8368339528315
iteration: 4/ residual: 1378.3977762653317
iteration: 5/ residual: 992.2924574159562
iteration: 6/ residual: 841.9114691101751
iteration: 7/ residual: 745.1075741178236
iteration: 8/ residual: 713.1971163543981
iteration: 9/ residual: 496.44442169031777
iteration: 10/ residual: 371.23657423415443
iteration: 11/ residual: 227.41513299183174
iteration: 12/ residual: 216.13875069724313
iteration: 13/ residual: 212.61689564343087
iteration: 14/ residual: 211.02641580538906
iteration: 15/ residual: 209.60974383999866
iteration: 16/ residual: 208.8142369153094
iteration: 17/ residual: 207.79105082062785
iteration: 18/ residual: 207.13265773674212
iteration: 19/ residual: 206.56270346338883
iteration: 20/ residual: 205.87012907799908
iteration: 21/ residual: 205.17438022765512
iteration: 22/ residual: 204.07185214416552
iteration: 23/ residual: 203.64823308845203
iter

In [14]:
using Plots

┌ Info: Recompiling stale cache file /home/jonatan/.julia/compiled/v1.0/PyPlot/oatAj.ji for PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1190
┌ Info: Installing matplotlib via the Conda matplotlib package...
└ @ PyCall /home/jonatan/.julia/packages/PyCall/ttONZ/src/PyCall.jl:705
┌ Info: Running `conda install -y matplotlib` in root environment
└ @ Conda /home/jonatan/.julia/packages/Conda/kLXeC/src/Conda.jl:112


Solving environment: ...working... done

# All requested packages already installed.



InitError: InitError: PyError (PyImport_ImportModule

The Python package matplotlib could not be found by pyimport. Usually this means
that you did not install matplotlib in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the matplotlib module, you can
use `pyimport_conda("matplotlib", PKG)`, where PKG is the Anaconda
package the contains the module matplotlib, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re-launch Julia.

) <class 'ImportError'>
ImportError("\n

In [ ]:
plotly()

In [ ]:
max = convert(Float64, 100.0)
ρs = collect(LinRange(1, 10, 10))
ks = zeros(0)

ρs2 = collect(LinRange(15, 100, 15))
ks2 = zeros(0)

for ρ = ρs
    println("Starting model $ρ")
    (c, x_s, tend, k) = admm_model_large(convert(Float64, ρ)) # admm_model_large(ρ)
    #styles = [:solid, :dash, :dot, :dashdot, :solid, :dash, :solid, :dash,]
    append!(ks, k)
end
plot(ρs, ks), 
    linewidth = 3,
    xlabel = "ρ",
    ylabel = "Iterations",
    #title  = "Penalty effect on number of iterations",
    size   = (600,400),
    reuse  = false,
    tickfontsize   = 8,
    legend = false,
    legendfontsize = 10,
    guidefontsize  = 10,
    grid = true)
scatter!(ρs, ks,show=true)
for ρ = ρs2
    println("Starting model $ρ")
    (c, x_s, tend, k) = admm_model_large(convert(Float64, ρ)) # admm_model_large(ρ)
    #styles = [:solid, :dash, :dot, :dashdot, :solid, :dash, :solid, :dash,]
    append!(ks2, k)
end
plot!(cat(ρs,ρs2, dims=1), cat(ks, ks2, dims=1), 
    linewidth = 3,
    xlabel = "ρ",
    ylabel = "Iterations",
    #title  = "Penalty effect on number of iterations",
    size   = (600,400),
    reuse  = false,
    tickfontsize   = 8,
    legend = false,
    legendfontsize = 10,
    guidefontsize  = 10,
    grid = true)
scatter!(cat(ρs,ρs2, dims=1), cat(ks, ks2, dims=1),show=true)
#savefig("performance_plot.pdf")

In [38]:
plot(ρs, ks, 
    linewidth = 3,
    xlabel = "ρ",
    ylabel = "Iterations",
    #title  = "Penalty effect on number of iterations",
    size   = (600,400),
    reuse  = false,
    tickfontsize   = 8,
    legend = false,
    legendfontsize = 10,
    guidefontsize  = 10,
    grid = true)
scatter!(ρs, ks)

<!DOCTYPE html>
 
 
 Plots.jl

In [42]:
ρs2 = collect(LinRange(1, 10, 10))
ks2 = zeros(0)

for ρ = ρs2
    println("Starting model $ρ")
    (c, x_s, tend, k) = admm_model_small(convert(Float64, ρ)) # admm_model_large(ρ)
    #styles = [:solid, :dash, :dot, :dashdot, :solid, :dash, :solid, :dash,]
    append!(ks2, k)
end

Starting model 1.0
iteration: 1/ residual: 139.38786052375087
iteration: 2/ residual: 23.687501982896663
iteration: 3/ residual: 10.078908131809321
iteration: 4/ residual: 5.5666515437871595
iteration: 5/ residual: 3.789498569682822
iteration: 6/ residual: 2.6198021507345337
iteration: 7/ residual: 2.292993021523727
iteration: 8/ residual: 1.3958068067987466
iteration: 9/ residual: 0.8233930303244121
iteration: 10/ residual: 0.6384427433023614
iteration: 11/ residual: 0.447422560729395
iteration: 12/ residual: 0.4030625075630626
iteration: 13/ residual: 0.23796661501170116
iteration: 14/ residual: 0.14446933666610165
iteration: 15/ residual: 0.11877970894605115
iteration: 16/ residual: 0.09898197820495536
Starting model 2.0
iteration: 1/ residual: 278.3974894635384
iteration: 2/ residual: 48.20642965571351
iteration: 3/ residual: 17.800089521283983
iteration: 4/ residual: 10.022405956065398
iteration: 5/ residual: 7.049378649398117
iteration: 6/ residual: 4.679177473710869
iteration: 7

iteration: 18/ residual: 0.5904573559518619
iteration: 19/ residual: 0.5779978059490394
iteration: 20/ residual: 0.5551908709277358
iteration: 21/ residual: 0.5321627298787639
iteration: 22/ residual: 0.5100678627083486
iteration: 23/ residual: 0.48994829899992454
iteration: 24/ residual: 0.47141045938134046
iteration: 25/ residual: 0.4349589687892769
iteration: 26/ residual: 0.3858342481627206
iteration: 27/ residual: 0.32616445792627474
iteration: 28/ residual: 0.28641468098678674
iteration: 29/ residual: 0.2772029510327458
iteration: 30/ residual: 0.25782878064312464
iteration: 31/ residual: 0.21964729714077763
iteration: 32/ residual: 0.19421840589214573
iteration: 33/ residual: 0.15156769986567478
iteration: 34/ residual: 0.11757260967896793
iteration: 35/ residual: 0.10984354267443593
iteration: 36/ residual: 0.10896504423191891
iteration: 37/ residual: 0.08210370329884528
Starting model 9.0
iteration: 1/ residual: 1252.1333934205288
iteration: 2/ residual: 219.2947102911723
iter

In [ ]:
ρs2 = collect(LinRange(1, 10, 10))
ks2 = zeros(0)

for ρ = ρs2
    println("Starting model $ρ")
    (c, x_s, tend, k) = admm_model_small(convert(Float64, ρ)) # admm_model_large(ρ)
    #styles = [:solid, :dash, :dot, :dashdot, :solid, :dash, :solid, :dash,]
    append!(ks2, k)
end

In [12]:
## Kill threads
rmprocs(procs())

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /buildworker/worker/package_linux32/build/usr/share/julia/stdlib/v1.0/Distributed/src/cluster.jl:932


Task (done) @0xd0fea590